In [92]:
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
import re

In [93]:
from fastai.tabular import add_datepart

In [94]:
df = pd.read_csv('../HackerEarthPredictGiftPrices/dataset/train.csv')

test = pd.read_csv('../HackerEarthPredictGiftPrices/dataset/test.csv')

In [95]:
df

,gift_id,gift_type,gift_category,gift_cluster,instock_date,stock_update_date,lsg_1,lsg_2,lsg_3,lsg_4,lsg_5,lsg_6,uk_date1,uk_date2,is_discounted,volumes,price
0,GF_11156,61,534,3942,2014-02-21 05:07:06.000,2016-11-09 15:49:51.000,3377,5221,504,1912,10,554,2014-02-24 08:07:06.000,2014-02-24 07:07:06.000,0,NaN,175.54
1,GF_11157,61,534,3942,2014-02-21 06:07:06.000,2016-11-11 13:49:51.000,3377,5221,504,1912,10,554,2014-02-22 07:07:06.000,2014-02-24 06:07:06.000,1,NaN,95.80
2,GF_15689,584,262,0,2014-02-21 09:30:21.000,2016-03-24 14:46:18.000,5290,1579,3203,1912,9,1578,2016-01-26 00:04:45.000,2016-03-18 02:00:00.000,1,NaN,107.35
3,GF_11155,61,534,3942,2014-02-22 05:07:06.000,2016-11-10 16:49:51.000,3377,5221,504,1912,10,554,2016-11-07 13:49:51.000,2016-11-06 04:00:00.000,0,NaN,172.90
4,GF_11158,61,534,3942,2014-02-22 07:07:06.000,2016-11-10 13:49:51.000,3377,5221,504,1912,9,554,2016-11-07 15:49:51.000,2016-11-06 01:00:00.000,1,NaN,77.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20274,GF_10269,105,704,6448,2016-11-12 13:46:42.000,2016-11-17 10:46:42.000,2055,6883,995,1912,10,1899,2016-11-14 14:46:42.000,2016-11-11 03:00:00.000,0,NaN,57.68
20275,GF_5854,1220,526,817,2016-11-12 13:46:47.000,2016-11-18 13:46:47.000,8323,6753,6706,1912,10,1899,2016-11-13 10:46:47.000,2016-10-28 02:00:00.000,0,NaN,122.87
20276,GF_563,509,705,821,2016-11-12 13:46:57.000,2017-01-21 19:30:04.000,2826,4009,2912,1912,10,1451,2017-01-21 18:30:04.000,2017-01-18 01:00:00.000,0,NaN,47.14
20277,GF_9107,682,213,5620,2016-11-12 13:47:01.000,2016-11-18 11:47:01.000,2089,6883,3607,1912,10,822,2016-11-14 12:47:01.000,2016-11-13 02:00:00.000,0,NaN,47.68


In [96]:
test

,gift_id,gift_type,gift_category,gift_cluster,instock_date,stock_update_date,lsg_1,lsg_2,lsg_3,lsg_4,lsg_5,lsg_6,uk_date1,uk_date2,is_discounted,volumes
0,GF_2372,842,663,24,2016-11-12 13:49:12.000,2016-11-16 10:49:12.000,203,1334,4358,1912,10,1899,2016-11-16 12:49:12.000,2016-11-13 04:00:00.000,0,NaN
1,GF_13040,407,534,754,2016-11-12 13:49:22.000,2016-11-17 14:49:22.000,4785,5940,2582,1912,10,1488,2016-11-12 14:49:22.000,2016-11-03 01:00:00.000,0,NaN
2,GF_5754,631,433,6012,2016-11-12 13:49:23.000,2016-11-16 13:49:23.000,1812,5027,3427,1912,10,1913,2016-11-15 11:49:23.000,2016-11-01 05:00:00.000,1,NaN
3,GF_1417,992,433,6358,2016-11-12 13:49:24.000,2016-11-16 11:49:24.000,2002,6883,5537,1912,10,1734,2016-11-14 12:49:24.000,2016-11-04 04:00:00.000,0,NaN
4,GF_23204,43,534,6123,2016-11-12 13:49:40.000,2016-11-18 11:49:40.000,1512,6883,610,1912,9,150,2016-11-14 10:49:40.000,2016-11-02 01:00:00.000,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13514,GF_29169,31,65,0,2017-04-07 20:51:34.000,2017-04-10 19:51:34.000,3252,3211,7565,1845,9,1899,2017-04-04 21:51:34.000,2017-03-30 07:00:59.623,0,27.0
13515,GF_29165,31,65,0,2017-04-07 22:51:34.000,2017-04-08 22:51:34.000,3252,3211,7565,1786,9,1899,2017-04-04 20:51:34.000,2017-04-02 09:00:59.623,0,23.0
13516,GF_29168,31,65,0,2017-04-07 22:51:34.000,2017-04-08 22:51:34.000,3252,3211,7565,1830,9,1899,2017-04-08 19:51:34.000,2017-03-31 06:00:59.623,0,23.0
13517,GF_33797,280,184,0,2017-04-15 18:55:17.000,2017-04-19 00:44:29.000,4198,3332,1805,1912,9,1899,2017-04-19 01:44:29.000,2017-04-15 18:03:15.352,0,NaN


In [97]:
df.describe()

,gift_type,gift_category,gift_cluster,lsg_1,lsg_2,lsg_3,lsg_4,lsg_5,lsg_6,is_discounted,volumes,price
count,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,20279.000000,7323.000000,20279.000000
mean,739.554662,394.171557,3303.358548,5314.595345,4187.653928,4866.945510,1679.152226,8.652695,1265.898171,0.229646,15.515363,143.404411
std,389.216989,235.077769,2541.082549,2703.317282,2274.875522,2713.856392,485.699119,2.349388,697.838495,0.420616,7.579669,267.281159
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,5.000000,0.010000
25%,403.000000,188.000000,587.000000,3311.000000,2251.000000,2548.000000,1801.000000,9.000000,577.500000,0.000000,9.000000,45.645000
50%,825.000000,433.000000,3231.000000,5520.000000,4246.000000,4839.000000,1912.000000,9.000000,1616.000000,0.000000,13.000000,75.600000
75%,1032.000000,534.000000,5787.000000,7535.000000,6504.500000,7387.000000,1912.000000,10.000000,1899.000000,0.000000,24.000000,126.845000
max,1360.000000,893.000000,7567.000000,9979.000000,7604.000000,9493.000000,2056.000000,10.000000,2065.000000,1.000000,29.000000,7010.270000


In [98]:
def missingcheck(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%']) #ptr
    return missing_data


In [99]:
missingcheck(df)

,Total,%
volumes,12956,63.9
price,0,0.0
lsg_2,0,0.0
gift_type,0,0.0
gift_category,0,0.0
gift_cluster,0,0.0
instock_date,0,0.0
stock_update_date,0,0.0
lsg_1,0,0.0
lsg_3,0,0.0


In [100]:
missingcheck(test)

,Total,%
volumes,2352,17.4
is_discounted,0,0.0
uk_date2,0,0.0
uk_date1,0,0.0
lsg_6,0,0.0
lsg_5,0,0.0
lsg_4,0,0.0
lsg_3,0,0.0
lsg_2,0,0.0
lsg_1,0,0.0


In [101]:
#dff = df.drop( ['volumes'],axis =1)
dff = df.fillna(0)
tff = test.fillna(0)

In [102]:
dff['instock_date'] = pd.to_datetime(dff['instock_date'] ,errors='coerce')
dff['justinstock_date'] = dff['instock_date'].dt.date


tff['instock_date'] = pd.to_datetime(tff['instock_date'] ,errors='coerce')
tff['justinstock_date'] = tff['instock_date'].dt.date

In [103]:
dff['stock_update_date'] = pd.to_datetime(dff['stock_update_date'] ,errors='coerce')
dff['juststock_update_date'] = dff['stock_update_date'].dt.date

tff['stock_update_date'] = pd.to_datetime(tff['stock_update_date'] ,errors='coerce')
tff['juststock_update_date'] = tff['stock_update_date'].dt.date

In [104]:
dff['uk_date1'] = pd.to_datetime(dff['uk_date1'] ,errors='coerce')
dff['justuk_date1'] = dff['uk_date1'].dt.date

tff['uk_date1'] = pd.to_datetime(tff['uk_date1'] ,errors='coerce')
tff['justuk_date1'] = tff['uk_date1'].dt.date

In [105]:
dff['uk_date2'] = pd.to_datetime(dff['uk_date2'] ,errors='coerce')
dff['justuk_date2'] = dff['uk_date2'].dt.date

tff['uk_date2'] = pd.to_datetime(tff['uk_date2'] ,errors='coerce')
tff['justuk_date2'] = tff['uk_date2'].dt.date

In [106]:
# no of days gift was in stock

In [107]:
dff['DaysStockUpdate'] = (dff['stock_update_date']- dff['instock_date']).dt.days


tff['DaysStockUpdate'] = (tff['stock_update_date']- tff['instock_date']).dt.days

In [108]:
#no of days between order1 and order2

In [109]:
dff['buyer12diff'] = (dff['uk_date1']-dff['uk_date2']).dt.days
dff['buyer12diff'] = np.absolute(dff['buyer12diff'])



tff['buyer12diff'] = (tff['uk_date1']-tff['uk_date2']).dt.days
tff['buyer12diff'] = np.absolute(tff['buyer12diff'])

In [110]:
# at order1 time was gift in stock

In [111]:
dff['buy1buytime'] = (dff['uk_date1'] -dff['instock_date']).dt.days

tff['buy1buytime'] = (tff['uk_date1'] -tff['instock_date']).dt.days

In [112]:
dff['buy1buytime'].sort_values()

tff['buy1buytime'].sort_values()



2658     -5
1107     -5
1259     -5
1260     -5
1269     -5
       ... 
967     137
941     142
936     143
1097    144
769     145
Name: buy1buytime, Length: 13519, dtype: int64

In [113]:
# order 2 in stock

In [114]:
dff['buy2buytime'] = (dff['uk_date2'] -dff['instock_date']).dt.days


tff['buy2buytime'] = (tff['uk_date2'] -tff['instock_date']).dt.days

In [115]:
dff['buy2buytime'].sort_values()

tff['buy2buytime'].sort_values()

2741   -668
2943   -664
3014   -663
2017   -661
1747   -660
       ... 
1097    134
811     134
46      137
936     137
769     138
Name: buy2buytime, Length: 13519, dtype: int64

In [116]:
# stockupdate date differece 

In [117]:
dff['stockupdatediff1'] = (dff['stock_update_date'] - dff['uk_date1']).dt.days


tff['stockupdatediff1'] = (tff['stock_update_date'] - tff['uk_date1']).dt.days

In [118]:
dff['stockupdatediff2'] = (dff['stock_update_date'] - dff['uk_date2']).dt.days


tff['stockupdatediff2'] = (tff['stock_update_date'] - tff['uk_date2']).dt.days

In [119]:
def add_datepart(df, fldname, drop=True):
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    
#     for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
#             'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
    
    for n in ('Year', 'Month', 'Week', 'Day','Hour', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[targ_pre+n] = getattr(fld.dt,n.lower())
    df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [120]:
add_datepart(dff, 'instock_date',drop = False)

add_datepart(tff, 'instock_date',drop = False)

In [121]:
add_datepart(dff, 'stock_update_date',drop =False)


add_datepart(tff, 'stock_update_date',drop =False)

In [122]:
add_datepart(dff, 'uk_date1', drop=False)


add_datepart(tff, 'uk_date1', drop=False)

In [123]:
add_datepart(dff, 'uk_date2', drop= False)


add_datepart(tff, 'uk_date2', drop= False)

In [124]:
dff['instock_date'].max()

tff['instock_date'].max()

Timestamp('2017-04-16 18:57:14')

In [125]:
dff['stock_update_date'].max()

tff['stock_update_date'].max()

Timestamp('2017-04-19 00:44:29')

In [126]:
dff['uk_date2'].max()

tff['uk_date2'].max()

Timestamp('2017-04-18 02:03:36.943000')

In [127]:
import holidays

In [128]:
### was it a holiday or not instockdate, stockupdateday,ukdate1,ukdate2
time = pd.DataFrame()

In [129]:
# start = dff['instock_date'].min()

# end = dff['stock_update_date'].max()

# time['date'] = pd.date_range(start,end)

In [130]:
# uk_holidays=[]
# for holiday in holidays.UK(years=[2014,2015,2016]).items():
#     uk_holidays.append(str(holiday[0]))

In [131]:
#uk_holidays

In [132]:
# dff['Isholidayinstock']= [ 1 if str(val).split()[0] in uk_holidays else 0 for val in dff['instock_date']]
# dff['Isholidaystockupdate']= [ 1 if str(val).split()[0] in uk_holidays else 0 for val in dff['stock_update_date']]
# dff['IsholidayUkdate1']= [ 1 if str(val).split()[0] in uk_holidays else 0 for val in dff['uk_date1']]
# dff['IsholidayUkdate2']= [ 1 if str(val).split()[0] in uk_holidays else 0 for val in dff['uk_date2']]

In [133]:
####### grace period dates ######

In [134]:
from datetime import datetime, timedelta

In [135]:
uk_graceholiday=[]
for holiday in holidays.UK(years=[2014,2015,2016]).items():
    uk_graceholiday.append(holiday[0])

In [136]:
graceondate=[]
for i in uk_graceholiday:
    twodayago = i - timedelta(days = 3)
    twodayafter = i + timedelta(days = 3)
    for date in pd.date_range(twodayago,twodayafter):
        graceondate.append((str(date).split()[0]))
    

In [137]:
uk_graceperiod = list(set(graceondate))

In [138]:
dff['Isholidayinstockgrceperiod']= [ 1 if str(val).split()[0] in uk_graceperiod else 0 for val in dff['instock_date']]
dff['Isholidaystock_update_dategrceperiod']= [ 1 if str(val).split()[0] in uk_graceperiod else 0 for val in dff['stock_update_date']]
dff['IsholidayUk_date1grceperiod']= [ 1 if str(val).split()[0] in uk_graceperiod else 0 for val in dff['uk_date1']]
dff['IsholidayUk_date2grceperiod']= [ 1 if str(val).split()[0] in uk_graceperiod else 0 for val in dff['uk_date2']]




tff['Isholidayinstockgrceperiod']= [ 1 if str(val).split()[0] in uk_graceperiod else 0 for val in tff['instock_date']]
tff['Isholidaystock_update_dategrceperiod']= [ 1 if str(val).split()[0] in uk_graceperiod else 0 for val in tff['stock_update_date']]
tff['IsholidayUk_date1grceperiod']= [ 1 if str(val).split()[0] in uk_graceperiod else 0 for val in tff['uk_date1']]
tff['IsholidayUk_date2grceperiod']= [ 1 if str(val).split()[0] in uk_graceperiod else 0 for val in tff['uk_date2']]


In [139]:
# interaction variables 

In [140]:
def createIntercations(data,cat_features):
    import itertools
    print(f"Creating features on {cat_features}, with combination 2 for training data /n")
    interactionstrain = pd.DataFrame(index=data.index)
    
    for col1 ,col2 in  itertools.combinations(cat_features,2):   
        newcolname = col1 + "_" + col2 #+"_" + col3
        new_values = data[col1].map(str) + "_" + data[col2].map(str) # + "_" + data[col3].map(str)
        interactionstrain[newcolname] = new_values

    print(interactionstrain)
    data_df = data.join(interactionstrain)
    
    
    print(data_df.shape)
    
    
    return data_df


In [141]:
def createIntercations2(data,cat_features):
    import itertools
    print(f"Creating features on {cat_features}, with combination 3 for training data /n")
    interactionstrain = pd.DataFrame(index=data.index)
    
    for col1 ,col2,col3 in  itertools.combinations(cat_features,3):   
        newcolname = col1 + "_" + col2 +"_" + col3
        new_values = data[col1].map(str) + "_" + data[col2].map(str)  + "_" + data[col3].map(str)
        interactionstrain[newcolname] = new_values

    print(interactionstrain)
    data_df = data.join(interactionstrain)
    
    
    print(data_df.shape)
    
    
    return data_df


In [142]:
# def createlogfestures(data,features):
  
#      cols = features
#      interactionstrain = pd.DataFrame(index=data.index)
        
#      for col  in cols:   
#             newcolname = col + "_log"   
#             new_values = np.log1p(data[col])
#             interactionstrain[newcolname] = new_values

  

#      data_df = data.join(interactionstrain)
  
#      print(data_df.shape)


#      return data_df 


In [143]:
# def createsqrtfeatures(train,test,features):
    

#   cols = features
#   interactionstrain = pd.DataFrame(index=train.index)
#   interactionstest = pd.DataFrame(index=test.index)
  
#   for col  in cols:   
#         newcolname = col + "_sqrt"   
#         new_values = np.sqrt(train[col])
#         interactionstrain[newcolname] = new_values

#   for col  in cols:   
#         newcolname = col + "_sqrt"   
#         new_values = np.sqrt(test[col])
#         interactionstest[newcolname] = new_values


#   train_df = train.join(interactionstrain)
#   test_df = test.join(interactionstest)

#   print(train_df.shape,test_df.shape)


#   return train_df ,test_df


In [144]:
interactiondff1 = createIntercations(dff,['gift_type','gift_category','gift_cluster'])

interactiontff1 = createIntercations(tff,['gift_type','gift_category','gift_cluster'])

Creating features on ['gift_type', 'gift_category', 'gift_cluster'], with combination 2 for training data /n
      gift_type_gift_category gift_type_gift_cluster  \
0                      61_534                61_3942   
1                      61_534                61_3942   
2                     584_262                  584_0   
3                      61_534                61_3942   
4                      61_534                61_3942   
...                       ...                    ...   
20274                 105_704               105_6448   
20275                1220_526               1220_817   
20276                 509_705                509_821   
20277                 682_213               682_5620   
20278                1061_212              1061_4987   

      gift_category_gift_cluster  
0                       534_3942  
1                       534_3942  
2                          262_0  
3                       534_3942  
4                       534_3942  
...     

In [145]:
interactiondff2 = createIntercations2(interactiondff1,['gift_type','gift_category','gift_cluster'])


interactiontff2 = createIntercations2(interactiontff1,['gift_type','gift_category','gift_cluster'])

Creating features on ['gift_type', 'gift_category', 'gift_cluster'], with combination 3 for training data /n
      gift_type_gift_category_gift_cluster
0                              61_534_3942
1                              61_534_3942
2                                584_262_0
3                              61_534_3942
4                              61_534_3942
...                                    ...
20274                         105_704_6448
20275                         1220_526_817
20276                          509_705_821
20277                         682_213_5620
20278                        1061_212_4987

[20279 rows x 1 columns]
(20279, 91)
Creating features on ['gift_type', 'gift_category', 'gift_cluster'], with combination 3 for training data /n
      gift_type_gift_category_gift_cluster
0                               842_663_24
1                              407_534_754
2                             631_433_6012
3                             992_433_6358
4            

In [146]:
interactiondff2.head()


interactiontff2.head()

,gift_id,gift_type,gift_category,gift_cluster,instock_date,stock_update_date,lsg_1,lsg_2,lsg_3,lsg_4,...,uk_date2Is_year_start,uk_date2Elapsed,Isholidayinstockgrceperiod,Isholidaystock_update_dategrceperiod,IsholidayUk_date1grceperiod,IsholidayUk_date2grceperiod,gift_type_gift_category,gift_type_gift_cluster,gift_category_gift_cluster,gift_type_gift_category_gift_cluster
0,GF_2372,842,663,24,2016-11-12 13:49:12,2016-11-16 10:49:12,203,1334,4358,1912,...,False,1479009600,0,0,0,0,842_663,842_24,663_24,842_663_24
1,GF_13040,407,534,754,2016-11-12 13:49:22,2016-11-17 14:49:22,4785,5940,2582,1912,...,False,1478134800,0,0,0,0,407_534,407_754,534_754,407_534_754
2,GF_5754,631,433,6012,2016-11-12 13:49:23,2016-11-16 13:49:23,1812,5027,3427,1912,...,False,1477976400,0,0,0,0,631_433,631_6012,433_6012,631_433_6012
3,GF_1417,992,433,6358,2016-11-12 13:49:24,2016-11-16 11:49:24,2002,6883,5537,1912,...,False,1478232000,0,0,0,0,992_433,992_6358,433_6358,992_433_6358
4,GF_23204,43,534,6123,2016-11-12 13:49:40,2016-11-18 11:49:40,1512,6883,610,1912,...,False,1478048400,0,0,0,0,43_534,43_6123,534_6123,43_534_6123


In [147]:
# import matplotlib.pyplot as plt
# interactiondff2.boxplot()

In [148]:
interactiondff3 = createIntercations(interactiondff2,['lsg_1','lsg_2','lsg_3','lsg_4','lsg_5','lsg_6'])

interactiontff3 = createIntercations(interactiontff2,['lsg_1','lsg_2','lsg_3','lsg_4','lsg_5','lsg_6'])

Creating features on ['lsg_1', 'lsg_2', 'lsg_3', 'lsg_4', 'lsg_5', 'lsg_6'], with combination 2 for training data /n
      lsg_1_lsg_2 lsg_1_lsg_3 lsg_1_lsg_4 lsg_1_lsg_5 lsg_1_lsg_6 lsg_2_lsg_3  \
0       3377_5221    3377_504   3377_1912     3377_10    3377_554    5221_504   
1       3377_5221    3377_504   3377_1912     3377_10    3377_554    5221_504   
2       5290_1579   5290_3203   5290_1912      5290_9   5290_1578   1579_3203   
3       3377_5221    3377_504   3377_1912     3377_10    3377_554    5221_504   
4       3377_5221    3377_504   3377_1912      3377_9    3377_554    5221_504   
...           ...         ...         ...         ...         ...         ...   
20274   2055_6883    2055_995   2055_1912     2055_10   2055_1899    6883_995   
20275   8323_6753   8323_6706   8323_1912     8323_10   8323_1899   6753_6706   
20276   2826_4009   2826_2912   2826_1912     2826_10   2826_1451   4009_2912   
20277   2089_6883   2089_3607   2089_1912     2089_10    2089_822   6883_

In [149]:
class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        from sklearn.preprocessing import LabelEncoder

        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)


In [150]:
def catVar2(data):
    categorical_colsT2 = [cname for cname in data.columns if data[cname].dtype == "object"]
    return categorical_colsT2


In [151]:
def robustlabelencoder(train,test):
    from sklearn.preprocessing import LabelEncoder

    label_enc = LabelEncoderExt()
    cols = catVar2(train)
    print(cols)
    for col in cols:
        label_enc.fit(train[col])
        train[col] = label_enc.transform(train[col])
        test[col] = label_enc.transform(test[col])
        
    print(train.shape,test.shape)
    
    return train,test


In [152]:
## delete dates

In [153]:
interactiondff3['justuk_date1'].dtypes


interactiontff3['justuk_date1'].dtypes

dtype('O')

In [154]:
date_cols = [cname for cname in interactiondff3.columns if interactiondff3[cname].dtype == "datetime64[ns]"]

In [155]:
date_cols

['instock_date', 'stock_update_date', 'uk_date1', 'uk_date2']

In [156]:
interactiondff3.shape

interactiontff3.shape

(13519, 105)

In [178]:
final_df = interactiondff3.drop(['instock_date', 'stock_update_date', 'uk_date1', 'uk_date2','justinstock_date','juststock_update_date','justuk_date1','justuk_date2','gift_id'],axis =1)*1


final_tf = interactiontff3.drop(['instock_date', 'stock_update_date', 'uk_date1', 'uk_date2','justinstock_date','juststock_update_date','justuk_date1','justuk_date2','gift_id'],axis =1)*1

In [179]:
final_df.columns

final_tf.columns

Index(['gift_type', 'gift_category', 'gift_cluster', 'lsg_1', 'lsg_2', 'lsg_3',
       'lsg_4', 'lsg_5', 'lsg_6', 'is_discounted', 'volumes',
       'DaysStockUpdate', 'buyer12diff', 'buy1buytime', 'buy2buytime',
       'stockupdatediff1', 'stockupdatediff2', 'instock_Year', 'instock_Month',
       'instock_Week', 'instock_Day', 'instock_Hour', 'instock_Dayofweek',
       'instock_Dayofyear', 'instock_Is_month_end', 'instock_Is_month_start',
       'instock_Is_quarter_end', 'instock_Is_quarter_start',
       'instock_Is_year_end', 'instock_Is_year_start', 'instock_Elapsed',
       'stock_update_Year', 'stock_update_Month', 'stock_update_Week',
       'stock_update_Day', 'stock_update_Hour', 'stock_update_Dayofweek',
       'stock_update_Dayofyear', 'stock_update_Is_month_end',
       'stock_update_Is_month_start', 'stock_update_Is_quarter_end',
       'stock_update_Is_quarter_start', 'stock_update_Is_year_end',
       'stock_update_Is_year_start', 'stock_update_Elapsed', 'uk_date1Year'

In [180]:
#feature selected finally 

dtrain = ['uk_date2Elapsed',
 'uk_date2Dayofyear',
 'uk_date2Day',
 'uk_date1Elapsed',
 'uk_date1Dayofyear',
 'stockupdatediff2',
 'stockupdatediff1',
 'stock_update_Hour',
 'stock_update_Elapsed',
 'stock_update_Dayofyear',
 'lsg_6',
 'lsg_5_lsg_6',
 'lsg_3_lsg_6',
 'lsg_3_lsg_5',
 'lsg_3_lsg_4',
 'lsg_3',
 'lsg_2_lsg_6',
 'lsg_2_lsg_5',
 'lsg_2_lsg_4',
 'lsg_2_lsg_3',
 'lsg_2',
 'lsg_1_lsg_3',
 'lsg_1_lsg_2',
 'lsg_1',
 'instock_Elapsed',
 'gift_type_gift_cluster',
 'gift_type_gift_category_gift_cluster',
 'gift_type_gift_category',
 'gift_type',
 'gift_cluster',
 'gift_category_gift_cluster',
 'gift_category',
 'buyer12diff',
 'buy2buytime',
 'DaysStockUpdate',
 'uk_date2Week',
 'uk_date1Week',
 'uk_date1Hour',
 'stock_update_Week',
 'stock_update_Day',
 'lsg_4_lsg_6',
 'lsg_1_lsg_5',
 'lsg_1_lsg_4',
 'is_discounted',
 'instock_Week',
 'instock_Hour',
 'instock_Dayofyear',
 'instock_Day',
 'buy1buytime',
 'volumes',
 'uk_date2Month',
 'uk_date2Hour',
 'uk_date2Dayofweek',
 'uk_date1Month',
 'uk_date1Is_quarter_end',
 'uk_date1Is_month_end',
 'uk_date1Dayofweek',
 'uk_date1Day',
 'stock_update_Dayofweek',
 'lsg_5',
 'lsg_4_lsg_5',
 'lsg_4',
 'lsg_1_lsg_6',
 'instock_Year',
 'instock_Month',
 'instock_Is_quarter_start',
 'instock_Is_quarter_end',
 'instock_Dayofweek',
 'Isholidayinstockgrceperiod',
 'IsholidayUk_date2grceperiod','price']

In [181]:
#feature selected finally 

dtest = ['uk_date2Elapsed',
 'uk_date2Dayofyear',
 'uk_date2Day',
 'uk_date1Elapsed',
 'uk_date1Dayofyear',
 'stockupdatediff2',
 'stockupdatediff1',
 'stock_update_Hour',
 'stock_update_Elapsed',
 'stock_update_Dayofyear',
 'lsg_6',
 'lsg_5_lsg_6',
 'lsg_3_lsg_6',
 'lsg_3_lsg_5',
 'lsg_3_lsg_4',
 'lsg_3',
 'lsg_2_lsg_6',
 'lsg_2_lsg_5',
 'lsg_2_lsg_4',
 'lsg_2_lsg_3',
 'lsg_2',
 'lsg_1_lsg_3',
 'lsg_1_lsg_2',
 'lsg_1',
 'instock_Elapsed',
 'gift_type_gift_cluster',
 'gift_type_gift_category_gift_cluster',
 'gift_type_gift_category',
 'gift_type',
 'gift_cluster',
 'gift_category_gift_cluster',
 'gift_category',
 'buyer12diff',
 'buy2buytime',
 'DaysStockUpdate',
 'uk_date2Week',
 'uk_date1Week',
 'uk_date1Hour',
 'stock_update_Week',
 'stock_update_Day',
 'lsg_4_lsg_6',
 'lsg_1_lsg_5',
 'lsg_1_lsg_4',
 'is_discounted',
 'instock_Week',
 'instock_Hour',
 'instock_Dayofyear',
 'instock_Day',
 'buy1buytime',
 'volumes',
 'uk_date2Month',
 'uk_date2Hour',
 'uk_date2Dayofweek',
 'uk_date1Month',
 'uk_date1Is_quarter_end',
 'uk_date1Is_month_end',
 'uk_date1Dayofweek',
 'uk_date1Day',
 'stock_update_Dayofweek',
 'lsg_5',
 'lsg_4_lsg_5',
 'lsg_4',
 'lsg_1_lsg_6',
 'instock_Year',
 'instock_Month',
 'instock_Is_quarter_start',
 'instock_Is_quarter_end',
 'instock_Dayofweek',
 'Isholidayinstockgrceperiod',
 'IsholidayUk_date2grceperiod']

In [182]:
final_df.isna().any().sum()

0

In [183]:
final_dfenc,final_tfenc = robustlabelencoder(final_df,final_tf)

['gift_type_gift_category', 'gift_type_gift_cluster', 'gift_category_gift_cluster', 'gift_type_gift_category_gift_cluster', 'lsg_1_lsg_2', 'lsg_1_lsg_3', 'lsg_1_lsg_4', 'lsg_1_lsg_5', 'lsg_1_lsg_6', 'lsg_2_lsg_3', 'lsg_2_lsg_4', 'lsg_2_lsg_5', 'lsg_2_lsg_6', 'lsg_3_lsg_4', 'lsg_3_lsg_5', 'lsg_3_lsg_6', 'lsg_4_lsg_5', 'lsg_4_lsg_6', 'lsg_5_lsg_6']
(20279, 97) (13519, 96)


In [184]:
final_tfenc.isna().any().sum()

0

In [185]:
# buy2buytime	7389	36.4
# stockupdatediff1	3885	19.2
# stockupdatediff2	3881	19.1
# buy1buytime

In [186]:
# final_dfenc['buy2buytime'] = np.absolute(final_dfenc['buy2buytime'])

# final_dfenc['stockupdatediff1'] = np.absolute(final_dfenc['stockupdatediff1'])

# final_dfenc['stockupdatediff2'] = np.absolute(final_dfenc['stockupdatediff2'])

# final_dfenc['buy1buytime'] = np.absolute(final_dfenc['buy1buytime'])

In [187]:
# final_tfenc['buy2buytime'] = np.absolute(final_tfenc['buy2buytime'])

# final_tfenc['stockupdatediff1'] = np.absolute(final_tfenc['stockupdatediff1'])

# final_tfenc['stockupdatediff2'] = np.absolute(final_tfenc['stockupdatediff2'])

# final_tfenc['buy1buytime'] = np.absolute(final_tfenc['buy1buytime'])

In [188]:
#numeric_feats = (final_dfenc.drop(['price'],axis=1)).dtypes[final_dfenc.dtypes != "object"].index

In [189]:
#numeric_feats = numeric_feats.drop(['buy2buytime','stockupdatediff1','stockupdatediff2','buy1buytime'])

In [190]:
#numeric_feats

In [79]:
# ############# skewnesss removal
# from scipy.stats import norm, skew #for some statistics


In [191]:
# #numeric_feats = (final_dfenc.drop(['price'],axis=1)).dtypes[final_dfenc.dtypes != "object"].index

# # Check the skew of all numerical features
# skewed_feats = final_dfenc[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
# print("\nSkew in numerical features: \n")
# skewness = pd.DataFrame({'Skew' :skewed_feats})
# skewness.head(10)

In [192]:
# skewness = skewness[abs(skewness) > 0.75]
# print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

# from scipy.special import boxcox1p
# skewed_features = skewness.index
# lam = 0.15
# for feat in skewed_features:
#     #all_data[feat] += 1
#     final_dfenc[feat] = boxcox1p(final_dfenc[feat], lam)

In [193]:
# skewness = skewness[abs(skewness) > 0.75]
# print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

# from scipy.special import boxcox1p
# skewed_features = skewness.index
# lam = 0.15
# for feat in skewed_features:
#     #all_data[feat] += 1
#     final_tf[feat] = boxcox1p(final_tf[feat], lam)

In [194]:
# train_data,fitted_lambda = stats.boxcox(train)

# # use lambda value to transform test data
# test_data = stats.boxcox(test, fitted_lambda)

In [195]:
# final_dfenc,final_tfenc = robustlabelencoder(final_df,final_tf)

In [196]:
final_dfencRed = final_dfenc[dtrain]


In [197]:
final_tfencRed = final_tfenc[dtest]

In [198]:
final_dfencRed.to_csv('finaltrainBOCO.csv',index = False)

In [199]:
final_tfencRed.to_csv('finaltestBOCO.csv',index = False)